In [1]:
# import the necessary libraries
import pandas as pd
import os

# set the path to the directory containing the Excel files
path = 'data'

# read in all the Excel files in the directory as a list of dataframes
dfs = [pd.read_excel(os.path.join(path, file)) for file in os.listdir(path) if file.endswith('.xlsx')]

# add a new column to each dataframe called 'pk' which should be the file name + index
for i, df in enumerate(dfs):
    df['pk'] = os.listdir(path)[i].split('.')[0] + '_' + df.index.astype(str)

# concatenate the list of dataframes into one big dataframe
master = pd.concat(dfs, ignore_index=True)

In [2]:
# # print out a list of all the column names
# print(master.columns)

In [3]:
# if the value of column 'code' in df master is 101 then ignore the row

In [4]:
# ____ DOB ____
# convert the 'Full DOB' column to a string
master['Full DOB'] = master['Full DOB'].astype(str)

# make a new column called 'cleaned_dob' and save the cleaned dob in that column
master['cleaned_dob'] = master['Full DOB']

# if len of 'Full DOB' is 4 then all we have is year and format it to '00/00/xxxx'
master['cleaned_dob'] = master['Full DOB'].apply(lambda dob: '00/00/' + dob if len(dob) == 4 else dob)

# convert the 'Full DOB' column to the same format the format is 'mmddyyyy'
master['cleaned_dob'] = master['Full DOB'].apply(lambda dob: dob[4:6] + '/' + dob[6:8] + '/' + dob[:4] if len(dob) == 8 else dob)

# define a function to check for matches
def check_match(row):
    cleaned_dob = row['cleaned_dob']
    inputs_dob = row['Inputs DOB']
    if cleaned_dob == 'nan':
        return 'no dob match'
    elif cleaned_dob == inputs_dob:
        return 'exact dob match'
    else:
        cleaned_dob_parts = cleaned_dob.split('/')
        inputs_dob_parts = inputs_dob.split('/')
        if len(cleaned_dob_parts) < 3 or len(inputs_dob_parts) < 3:
            return 'no dob match'
        else:
            match_types = {
                'year and month match': [2, 0],
                'year and day match': [2, 1],
                'month and day match': [0, 1],
                'year match': [2],
                'month match': [0],
                'day match': [1],
                'no dob match': []
            }
            for match_type, match_indices in match_types.items():
                if all(cleaned_dob_parts[idx] == inputs_dob_parts[idx] for idx in match_indices):
                    return match_type
            else:
                return 'no dob match'

# apply the function to each row of the dataframe
master['dob_match_finding'] = master.apply(check_match, axis=1)

# valuecounts of column 'dob_match_finding'
print(master['dob_match_finding'].value_counts())

year match              11116
exact dob match          8807
no dob match             2270
month and day match      1136
year and month match      512
Name: dob_match_finding, dtype: int64


In [5]:
from fuzzywuzzy import fuzz

# apply string, strip, and lower operations to each column
master[['Inputs First Name', 'Inputs Middle Name', 'Inputs Last Name', 'First Name', 'Middle Name', 'Last Name']] = master[['Inputs First Name', 'Inputs Middle Name', 'Inputs Last Name', 'First Name', 'Middle Name', 'Last Name']].apply(lambda x: x.astype(str).str.strip().str.lower())

def check_fuzzy_match(row):
    cleaned_first_name = row['Inputs First Name']
    cleaned_middle_name = row['Inputs Middle Name']
    cleaned_last_name = row['Inputs Last Name']
    returned_first_name = row['First Name']
    returned_middle_name = row['Middle Name']
    returned_last_name = row['Last Name']
    if pd.isna(cleaned_first_name) or pd.isna(cleaned_last_name):
        return 'no name match'
    # the matches should be in a descending order so the full name match should be first. as you go down the list the matches should become more granular.
    match_types = {
        'full name match': cleaned_first_name == returned_first_name and cleaned_last_name == returned_last_name,
        'last and middle name match': cleaned_last_name == returned_last_name and cleaned_middle_name == returned_middle_name,
        'first and last name match': cleaned_first_name == returned_first_name and cleaned_last_name == returned_last_name,
        'first and middle name match': cleaned_first_name == returned_first_name and cleaned_middle_name == returned_middle_name,
        'first name match': cleaned_first_name == returned_first_name,
        'last name match': cleaned_last_name == returned_last_name,
        'middle name match': cleaned_middle_name == returned_middle_name,
    }
    for match_type, condition in match_types.items():
        if condition:
            return match_type
    # calculate the fuzzy match score for the first name and last name
    first_name_score = fuzz.token_sort_ratio(cleaned_first_name, returned_first_name)
    last_name_score = fuzz.token_sort_ratio(cleaned_last_name, returned_last_name)
    # set a threshold for the fuzzy match score
    threshold = 80
    if first_name_score >= threshold and last_name_score >= threshold:
        # check which name part has a partial match and return that part
        if cleaned_first_name.lower() in returned_first_name.lower():
            return f'partial match first name'
        elif cleaned_middle_name.lower() in returned_middle_name.lower():
            return f'partial match middle name'
        elif cleaned_last_name.lower() in returned_last_name.lower():
            return f'partial match last name'
    return 'no name match'

# apply the function to each row of the dataframe
master['name_match_findings'] = master.apply(check_fuzzy_match, axis=1)

# valuecounts of column 'name_match_findings'
print(master['name_match_findings'].value_counts())

# the sum of the valuecounts should be equal to the number of rows in the dataframe
print(master['name_match_findings'].value_counts().sum())



full name match                11938
last name match                 7054
last and middle name match      1777
no name match                   1536
middle name match                994
first name match                 354
first and middle name match      169
partial match last name           17
partial match first name           2
Name: name_match_findings, dtype: int64
23841


In [6]:
# combine the two findings into a new column called 'combined_match_findings' they should be items in a list
master['combined_match_findings'] = master[['dob_match_finding', 'name_match_findings']].values.tolist()

# valuecounts of column 'combined_match_findings'
print(master['combined_match_findings'].value_counts())

[exact dob match, full name match]                     8034
[year match, last name match]                          6168
[year match, full name match]                          2888
[year match, last and middle name match]               1559
[no dob match, no name match]                          1327
[no dob match, middle name match]                       940
[month and day match, full name match]                  683
[exact dob match, last name match]                      408
[month and day match, last name match]                  352
[year and month match, full name match]                 331
[year match, first name match]                          214
[year match, no name match]                             160
[exact dob match, last and middle name match]           143
[year and month match, last name match]                 126
[exact dob match, first name match]                     125
[year match, first and middle name match]                79
[exact dob match, first and middle name 

In [7]:
# write the excel file to hard drive
writer = pd.ExcelWriter('turo_ncis.xlsx', engine='xlsxwriter')
master.to_excel(writer, sheet_name='master', index=False)
# create  second sheet with valuecounts of 'combined_match_findings'
master['combined_match_findings'].value_counts().to_excel(writer, sheet_name='valuecounts')
writer.save()

# close the writer
writer.close()
